# Predecir el precio de casas

# Explorando y procesando nuestros datos

In [0]:
import pandas as pd

In [0]:
! git clone https://github.com/Carlos-Trinidad/predict-house-price.git

In [0]:
! ls

In [0]:
df = pd.read_csv('predict-house-price/housepricedata.csv')

In [0]:
df

El dataset que tenemos ahora es lo que se conoce como un pandas dataframe. Para convertirlo en un array, simplemente accedemos a sus valores:

In [0]:
dataset = df.values

In [0]:
dataset

Ahora, dividimos el conjunto de datos en input features y la label que deseamos predecir.

In [0]:
X = dataset[:,0:10]
Y = dataset[:,10]

La normalización de nuestros datos es muy importante, ya que queremos que las características de entrada (input features) estén en el mismo orden de magnitud para facilitar nuestro entrenamiento. Usaremos un escalador min-max de scikit-learn que escala nuestros datos para que estén entre 0 y 1.

In [0]:
from sklearn import preprocessing

In [0]:
min_max_scaler = preprocessing.MinMaxScaler()
X_scale = min_max_scaler.fit_transform(X)

In [0]:
X_scale

Por último, deseamos reservar algunas partes de nuestro conjunto de datos para un conjunto de validación y un conjunto de prueba. Usamos la función train_test_split de scikit-learn para realizar esto.

In [0]:
from sklearn.model_selection import train_test_split

In [0]:
X_train, X_val_and_test, Y_train, Y_val_and_test = train_test_split(X_scale, Y, test_size=0.3)

In [0]:
X_val, X_test, Y_val, Y_test = train_test_split(X_val_and_test, Y_val_and_test, test_size=0.5)

In [0]:
print(X_train.shape, X_val.shape, X_test.shape, Y_train.shape, Y_val.shape, Y_test.shape)

# Creando y entrenando nuestra red neuronal



Usaremos Keras para construir nuestra arquitectura. Importemos de Keras directamente el código que necesitaremos usar:

In [0]:
from keras.models import Sequential
from keras.layers import Dense

Usaremos el modelo Sequential, lo que significa que simplemente necesitamos describir las capas de arriba en secuencia. Nuestra red neuronal tiene tres capas:

- Hidden layer 1: 30 neurons, ReLU activation
- Hidden layer 2: 30 neurons, ReLU activation
- Output Layer: 1 neuron, Sigmoid activation

In [0]:
model = Sequential([
    Dense(32, activation='relu', input_shape=(10,)),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid'),
])

Ahora que tenemos nuestra arquitectura especificada, necesitamos encontrar los mejores números para ella. Antes de comenzar nuestro entrenamiento, tenemos que configurar el modelo por
- Decirle qué algoritmo desea usar para hacer la optimización (utilizaremos stochastic gradient descent)
- Decirle qué función de pérdida usar (loss function) (para la clasificación binaria, usaremos binary cross entropy)
- Decirle qué otras métricas desea rastrear aparte de la función de pérdida (también queremos rastrear la accuaracy)

Lo hacemos a continuación:

In [0]:
model.compile(optimizer='sgd',
              loss='binary_crossentropy',
              metrics=['accuracy'])

Para entrenar nuestro módelo con nuestros datos es bastante sencillo y requiere escribir solo una línea de código. La funcion se llama "fit" y en ella específicaremos:
- que datos estamos entrenando, los cuales son X_train y Y_train
- el tamaño de nuestro mini-batch
- por cuanto tiempo queremos entrenarlo (epochs)
- cuáles son nuestros datos de validación para que el modelo nos diga cómo lo estamos haciendo en los datos de validación de cada punto.

Esta función generará un historial, que guardaremos bajo la variable hist. Usaremos esta variable un poco más adelante.

In [0]:
hist = model.fit(X_train, Y_train,
          batch_size=32, epochs=100,
          validation_data=(X_val, Y_val))

Evaluamos nuestro modelo con nuestro test set:

In [0]:
model.evaluate(X_test, Y_test)[1]

La razón por la que tenemos el índice 1 después de la función model.evaluate es porque la función devuelve el loss como el primer elemento y el accuracy como el segundo elemento. Para obtener solo la precisión (accuracy), simplemente accedemos al segundo elemento (que está indexado por 1, ya que el primer elemento comienza su indexación desde 0).

A continuación podemos ver cómo nos devolvería los dos valores:

In [0]:
model.evaluate(X_test, Y_test)

Por último, también podemos ver como es la predicción de nuestro modelo con cada uno de nuestros datos de test:

In [0]:
model.predict(X_test)

# Visualizando el Loss y Accuracy

Importamos el paquete relevante que necesitamos para hacer la visualización

In [0]:
import matplotlib.pyplot as plt

Queremos visualizar la pérdida de entrenamiento (training loss) y la pérdida de validación (validation loss) de esta manera:

In [0]:
plt.plot(hist.history['loss'])
plt.plot(hist.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Val'], loc='upper right')
plt.show()

También podemos visualizar la precisión del entrenamiento (training accuracy) y la precisión de la validación (validation accuracy) de esta manera:

In [0]:
plt.plot(hist.history['acc'])
plt.plot(hist.history['val_acc'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Val'], loc='lower right')
plt.show()